In [1]:
from openai import OpenAI
import pandas as pd
import time
import os
from dotenv import load_dotenv
# import requests

In [2]:
# Load API Key dari .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# === [2] Baca file CSV ===
df_ItBRI = pd.read_csv('../../dataset/data_ItBRI.csv')
df_ItPLN = pd.read_csv('../../dataset/data_ItPLN.csv')
df_ITTS = pd.read_csv('../../dataset/data_ITTS.csv')
df_Tel_U = pd.read_csv('../../dataset/data_Tel-U.csv')
df_UISI = pd.read_csv('../../dataset/data_UISI.csv')
df_ULBI = pd.read_csv('../../dataset/data_ULBI.csv')
df_Uper = pd.read_csv('../../dataset/data_Uper.csv')


In [3]:
# Cek jumlah baris dan kolom
print("Data ItBRI: " + str(df_ItBRI.shape[0]) + " rows, " + str(df_ItBRI.shape[1]) + " columns")
print("Data ItPLN: " + str(df_ItPLN.shape[0]) + " rows, " + str(df_ItPLN.shape[1]) + " columns")
print("Data ITTS: " + str(df_ITTS.shape[0]) + " rows, " + str(df_ITTS.shape[1]) + " columns")
print("Data Tel-U: " + str(df_Tel_U.shape[0]) + " rows, " + str(df_Tel_U.shape[1]) + " columns")
print("Data UISI: " + str(df_UISI.shape[0]) + " rows, " + str(df_UISI.shape[1]) + " columns")
print("Data ULBI: " + str(df_ULBI.shape[0]) + " rows, " + str(df_ULBI.shape[1]) + " columns")
print("Data Uper: " + str(df_Uper.shape[0]) + " rows, " + str(df_Uper.shape[1]) + " columns")

Data ItBRI: 78 rows, 9 columns
Data ItPLN: 188 rows, 9 columns
Data ITTS: 803 rows, 9 columns
Data Tel-U: 988 rows, 9 columns
Data UISI: 188 rows, 9 columns
Data ULBI: 596 rows, 9 columns
Data Uper: 803 rows, 9 columns


In [4]:
print(df_UISI.columns)

Index(['page', 'name', 'link', 'thumbnail', 'rating', 'date', 'snippet',
       'images', 'local_guide'],
      dtype='object')


In [14]:
# === Load data CSV ===
df_UISI = pd.read_csv("../../dataset/data_UISI.csv")
df_UISI['label_sentimen'] = None  # Tambahkan kolom baru

# === Fungsi klasifikasi sentimen ===
def label_sentimen(snippet):
    try:
        prompt = f"""
                    Tolong klasifikasikan sentimen dari review berikut ke dalam salah satu dari tiga label: positif, negatif, atau netral.

                    Review: "{snippet}"

                    Jawaban hanya satu kata: positif / negatif / netral.
                    """
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # atau gpt-4
            messages=[
                {"role": "system", "content": "Kamu adalah asisten untuk klasifikasi sentimen review kampus."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
        )
        label = response.choices[0].message.content.strip().lower()

        if label in ['positif', 'negatif', 'netral']:
            return label
        else:
            return 'netral'
    except Exception as e:
        print(f"Terjadi error: {e}")
        return 'netral'

In [15]:

# === Loop untuk melabeli setiap review ===
for i, row in df_UISI.iterrows():
    if pd.notnull(row['snippet']):
        print(f"Labeling review ke-{i+1}...")
        df_UISI.at[i, 'label_sentimen'] = label_sentimen(row['snippet'])
        time.sleep(1.5)  # Hindari rate limit


Labeling review ke-1...
Labeling review ke-2...
Labeling review ke-3...
Labeling review ke-4...
Labeling review ke-5...
Labeling review ke-6...
Labeling review ke-7...
Labeling review ke-8...
Labeling review ke-9...
Labeling review ke-10...
Labeling review ke-11...
Labeling review ke-12...
Labeling review ke-13...
Labeling review ke-14...
Labeling review ke-15...
Labeling review ke-16...
Labeling review ke-17...
Labeling review ke-18...
Labeling review ke-19...
Labeling review ke-20...
Labeling review ke-21...
Labeling review ke-22...
Labeling review ke-23...
Labeling review ke-24...
Labeling review ke-25...
Labeling review ke-26...
Labeling review ke-27...
Labeling review ke-28...
Labeling review ke-29...
Labeling review ke-30...
Labeling review ke-31...
Labeling review ke-32...
Labeling review ke-33...
Labeling review ke-34...
Labeling review ke-35...
Labeling review ke-36...
Labeling review ke-37...
Labeling review ke-38...
Labeling review ke-39...
Labeling review ke-40...
Labeling 

In [16]:

# Simpan ke file baru
output_dir = '../../dataset/labeled_data/'
os.makedirs(output_dir, exist_ok=True)
df_UISI.to_csv(os.path.join(output_dir, 'UISI_labelled.csv'), index=False)
print("Labeling selesai. File hasil disimpan.")

Labeling selesai. File hasil disimpan.
